<a href="https://colab.research.google.com/github/marinasemushina/-_-3/blob/main/DL_HW1_SemushinaMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.utils import to_categorical

In [13]:
# Загрузка данных
train_data = pd.read_csv('fmnist_train.csv')
test_data = pd.read_csv('fmnist_test.csv')

# Проверка структуры данных
print("Структура обучающего набора:")
print(train_data.head())

# Удаление NaN значений
train_data_cleaned = train_data.dropna()

# Подготовка данных
X = train_data_cleaned.drop(columns=['label', 'Id']).values.astype('float32') / 255.0  # Нормализация
y = train_data_cleaned['label'].values  # Метки классов

# One-hot кодирование меток
y = to_categorical(y, num_classes=10)

# Разделение данных на обучающую и валидационную выборки
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Проверка формы данных
print("Форма обучающего набора:", X_train.shape)
print("Форма валидационного набора:", X_val.shape)

Структура обучающего набора:
   label  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  \
0      2       0       0       0       0       0       0       0       0   
1      9       0       0       0       0       0       0       0       0   
2      6       0       0       0       0       0       0       0       5   
3      0       0       0       0       1       2       0       0       0   
4      3       0       0       0       0       0       0       0       0   

   pixel9  ...  pixel776  pixel777  pixel778  pixel779  pixel780  pixel781  \
0       0  ...       0.0       0.0       0.0       0.0       0.0       0.0   
1       0  ...       0.0       0.0       0.0       0.0       0.0       0.0   
2       0  ...       0.0       0.0      30.0      43.0       0.0       0.0   
3       0  ...       0.0       0.0       0.0       0.0       1.0       0.0   
4       0  ...       0.0       0.0       0.0       0.0       0.0       0.0   

   pixel782  pixel783  pixel784  Id  
0      

In [14]:
# Изменение формы данных для CNN
X_train = X_train.reshape(-1, 28, 28, 1)
X_val = X_val.reshape(-1, 28, 28, 1)

# Подготовка тестового набора
X_test = test_data.drop(columns=['Id']).values.astype('float32') / 255.0
X_test = X_test.reshape(-1, 28, 28, 1)

# Построение модели CNN
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
# Компиляция модели
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Обучение модели
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=128)

Epoch 1/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 14s 108ms/step - accuracy: 0.5522 - loss: 1.2848 - val_accuracy: 0.7817 - val_loss: 0.5852
Epoch 2/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 20s 107ms/step - accuracy: 0.7984 - loss: 0.5352 - val_accuracy: 0.8283 - val_loss: 0.4781
Epoch 3/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 20s 102ms/step - accuracy: 0.8360 - loss: 0.4423 - val_accuracy: 0.8383 - val_loss: 0.4474
Epoch 4/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 20s 102ms/step - accuracy: 0.8557 - loss: 0.3941 - val_accuracy: 0.8427 - val_loss: 0.4299
Epoch 5/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 20s 95ms/step - accuracy: 0.8667 - loss: 0.3597 - val_accuracy: 0.8574 - val_loss: 0.4013
Epoch 6/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 21s 102ms/step - accuracy: 0.8724 - loss: 0.3401 - val_accuracy: 0.8694 - val_loss: 0.3693
Epoch 7/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 11s 104ms/step - accuracy: 0.8808 - loss: 0.3244 - val_accuracy: 0.8762 - val_loss: 0.3468
Epoch 8/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 21s 104ms/step - accuracy: 0.8867 - loss: 0.

In [16]:
# Предсказание на тестовом наборе
predictions = model.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step


In [17]:
# Подготовка файла для отправки
submission = pd.DataFrame({'id': test_data['Id'], 'label': predicted_classes})
submission.to_csv('submission.csv', index=False)